### use LM 

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [ ]:
train = pd.read_csv('./train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./test.csv', encoding = 'utf-8-sig')

In [ ]:
FINETUNE_MODEL = "./finetune_weight_1"
model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map= {"":0})

In [ ]:
model_id = 'beomi/gemma-ko-7b'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
def query_to_prompt(query):
  messages = [
      {
          "role": "system",
          "content": (
              "You are a helpful assistant specializing in restoring obfuscated Korean reviews. "
              "Your task is to transform the given obfuscated Korean review into a clear, correct, "
              "and natural-sounding Korean review that reflects its original meaning. "
              "Spacing and word length in the output must be restored to the same as in the input. "
              "Do not provide any description. Print only in Korean."
          )
      },
      {
          "role": "user",
          "content": f"input : {query}, output : "
      },
  ]

  prompt = "\n".join([m["content"] for m in messages]).strip()

  return prompt

In [ ]:
def restore_review(query):
  prompt = query_to_prompt(query)
  query_len = len(query)

  outputs = pipe(
      prompt,
      do_sample=True,
      temperature=0.2,
      top_p=0.9,
      max_new_tokens=len(query),
      eos_token_id=pipe.tokenizer.eos_token_id
  )

  generated_text = outputs[0]['generated_text']
  result = generated_text[len(prompt):].strip()

  # clean
  result = result.split("'")[0]
  result = result.split("\n")[0]
  if 'output' in result:
    result = result[:query_len]

  return result

In [ ]:
restored_reviews = []

print(len(train))

for index, row in train.iterrows():
    print(f"index: {index}")

    query = row['input']
    query_len = len(query)

    result = restore_review(query)

    restored_reviews.append(result)

In [ ]:
train['restored'] = restored_reviews

In [ ]:
train.to_csv('./train_augmented.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
restored_reviews

---

### use konoise

In [1]:
%%capture
!pip install --upgrade pip setuptools
!pip install setuptools-rust
!pip install tqdm

In [2]:
!pip install konoise==1.0.4.6

In [3]:
%%capture
!pip install pandas

In [1]:
from konoise import NoiseGenerator
import pandas as pd
import time
import concurrent.futures

In [2]:
train = pd.read_csv('./train_augmented.csv', encoding = 'utf-8-sig')

In [3]:
def noiser(text, methods='change-vowels'): 
    generator = NoiseGenerator(num_cores=1)
    text = generator.generate(
        text, 
        methods=methods,  
        prob=1., 
        delimeter='sentence'
    )
    # time.sleep(0.1)

    return text 

In [ ]:
noised_reviews = []

for index, row in train.iterrows():
    print(f"index: {index}")
    if index in [80, 173, 649, 795, 1738, 1948, 246]:
        continue

    text = row['output']
    result = noiser(text, 'change-vowels')
    
    noised_reviews.append(result)

index: 0


100%|██████████| 4/4 [00:00<00:00, 3642.47it/s]

index: 1



100%|██████████| 2/2 [00:00<00:00, 66052.03it/s]

index: 2


...


100%|██████████| 2/2 [00:00<00:00, 48210.39it/s]

index: 2462



100%|██████████| 1/1 [00:00<00:00, 5841.65it/s]


index: 2463


100%|██████████| 1/1 [00:00<00:00, 59918.63it/s]


In [ ]:
train['noised'] = noised_reviews

In [6]:
noised_reviews

['별 한 개됴 야깝댜. 왜 샤람들이 별 1개를 쥬는지 겪여본 샤람으료셔 말료 설명햐쟈니 댓글료는 녀뮤 길교... 야뮤튼 듀 번 댜시 갸길 싫은 곳. 캠핑을 20어 년 댜너본 곳 중 제일 기분 냐빴던 곳.',
 '잠만 쟈교 갈 때 좋네오. 잠옷됴 줌 ㅋ',
 '절대 갸면 안 되는 곳 메묘',
 '야... 갸격 좋교 부됴 뻥 뚫러셔 시원햐지만 담배 냄새 미처벼림. 쌰게 햐류만 묵겠댜! 햐는 샤람한테만 츄천. 담배 냄새갸 묘든 장점을 갸저갸는 곳. 뇨래방에셔 각종 담배와 우흥에 쩔었을 때 냐는 냄새갸 계속 방에 있음 ㅆ... 쌰니꺄 할 말 없음.',
 '지인 츄천으료 댜너왔는데 갸성비 좋교 깔끔햐네오. 져렴한데 물과 음로됴 있교 방됴 야쥬 땨뜻했습니댜. 휘닉스퍄크와 겨리됴 멀지 않야셔 좋았교오. 뱌료 옆에 편의점이 있여셔 편리했습니댜. 댜음에됴 이용할 슈 있으면 이용햐교 싶네오. 예약이 여렵지만 ^^',
 '편안히 잘 쉬교 왔습니댜. 쥬위에 맛집됴 많야셔 좋은 것 같야오.',
 '스위트룸의 값여치갸 있네 ㅋ 교급 침규에 킹베드, 햐벼부 욕죠, 캬페 같은 티 테이블, 넓은 쇼퍄에 겨실, 부, 냉장교, 죠식당꺄지 묘든 게 완벽했댜.',
 '재방문 의샤 전허 없음. 신랴스테이뵤댜 뎌 작음. 효텔 쥬챠장 15대만 갸능. 효텔 1일 쥬챠 오금 2만 원. 댜른 우로 쥬챠장됴 2만 원. 효텔 남직원 불친절.',
 '녀뮤 멋지교 공기 좋습니댜. 최교입니댜!',
 '장점: 청결함, 쥬챠 시설 좋음, 시설 좋음.   단점: 쥬챠 출챠할 때 불편함, 룸 셔비스 없음, 쥬변에 뭐갸 없여셔 샤려 갸기 불편함, 해당 층에 뭐갸 있는지 영여료 쎠 있여 불편함, 죠식이 생각뵤댜 별료임.',
 '뻔히 30분 넘게 신효 받야셔 기댜리는 줄 봤으면셔 말 안햐교 있댜갸 거유 좌회전을 받야셔 들여갸니 쥬챠장 입규셔 만챠랴교 뱌료 돌러뵤냅니댜. 기댜리면 안되나 해됴 그냥 댜 돌러뵤냅니댜. 밑에 기댜리는 샤람들은 져만큼 냐요니 쥬챠할 슈 있을 겨랴교 생각햐지 그게 댜 들여갸뵤지됴 못햐교 그냥 뺘뀨당한 챠

In [ ]:
train.to_csv('./train_augmented.csv', index = False, encoding = 'utf-8-sig')